In [ ]:
from ophyd import EpicsMotor
m1 = EpicsMotor("xxx:m1", name="m1")

In [ ]:
m1.position

In [ ]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("xxx:scaler1", name="scaler")

In [ ]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

In [ ]:
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels

scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")

scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

In [ ]:
RE(bp.scan([scaler], m1, -1, 1, 5))